In [11]:
# 지연 통계 : 데이터 전처리
import pandas as pd

col = ['공항명', '출/도착', '운항', '기상', 'A/C접속', 'A/C정비', '여객처리', '복합원인', '기타', '계']
# 인천공항 
incheon = pd.read_csv("C:/Data/airport/지연통계_인천공항.csv")
print("[인천공항 원본 데이터]")
incheon

[인천공항 원본 데이터]


,기상,A/C접속,A/C정비,여객처리 및 승무원관련,복합원인,기타,운항(편),총 지연편수
0,1834,34174,3719,1130,619,23367,1432905,64843


In [12]:
# 열 이름 변경
incheon.columns = ['기상', 'A/C접속', 'A/C정비', '여객처리', '복합원인', '기타', '운항', '계']

# 공항명 열 추가
incheon['공항명'] = '인천'

# 열 순서 변경
incheon = incheon[['공항명', '운항', '기상', 'A/C접속', 'A/C정비', '여객처리', '복합원인', '기타', '계']]

print("[인천공항 전처리 데이터]")
incheon

[인천공항 전처리 데이터]


,공항명,운항,기상,A/C접속,A/C정비,여객처리,복합원인,기타,계
0,인천,1432905,1834,34174,3719,1130,619,23367,64843


In [13]:
# 국내공항(인천공항 제외)
other = pd.read_csv("C:/Data/airport/지연통계_국내공항.csv", encoding = 'cp949')

print("[국내공항(인천 제외) 원본 데이터]")
other.head(10)

[국내공항(인천 제외) 원본 데이터]


,Unnamed: 0,Unnamed: 1,Unnamed: 2,지연 통계,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,검색 기간,2017.01~2021.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,노선 구분,전체,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,공항명,출/도착,운항(편),기상,A/C접속,A/C장비,여객처리,복합원인,기타,계
5,김포,출발,339517,1633,29149,1037,885,936,2655,36295
6,김포,도착,339927,477,13552,355,113,67,290,14854
7,김포,계,679444,2110,42701,1392,998,1003,2945,51149
8,김해,출발,220121,611,10775,487,532,184,914,13503
9,김해,도착,220286,890,5989,304,74,33,953,8243


In [14]:
# 기간, 노선 구분을 나타내는 행 제거
other = other.drop([0, 1, 2, 3, 4])

# 열 이름 변경(공통되는 열 이름)
other.columns = col

# 결측값 처리 
other = other.dropna(thresh = 9)

# 합계값만 필터링
mask = other['출/도착'] == '계'
other = other[mask]

# 출/도착 열 삭제
other = other.drop(['출/도착'], axis = 1)

print("[국내공항(인천 제외) 전처리 데이터]")
other.head()

[국내공항(인천 제외) 전처리 데이터]


,공항명,운항,기상,A/C접속,A/C정비,여객처리,복합원인,기타,계
7,김포,679444,2110,42701,1392,998,1003,2945,51149
10,김해,440407,1501,16764,791,606,217,1867,21746
13,제주,809463,3221,86033,1244,598,334,1739,93169
16,대구,107511,303,5578,89,44,7,307,6328
19,광주,67860,481,5258,74,38,79,168,6098


In [15]:
# 두 데이터프레임 병합(인천 + 국내공항(인천제외))
airports = pd.concat([incheon, other])
airports = airports.reset_index()
airports.drop('index', axis = 1, inplace = True)
print("[국내공항 데이터]")
airports.head()

[국내공항 데이터]


,공항명,운항,기상,A/C접속,A/C정비,여객처리,복합원인,기타,계
0,인천,1432905,1834,34174,3719,1130,619,23367,64843
1,김포,679444,2110,42701,1392,998,1003,2945,51149
2,김해,440407,1501,16764,791,606,217,1867,21746
3,제주,809463,3221,86033,1244,598,334,1739,93169
4,대구,107511,303,5578,89,44,7,307,6328


In [16]:
# 데이터 타입 변경
airports = airports.astype({'운항' : 'int', '기상' : 'int', 'A/C접속' : 'int', 
                 'A/C정비' : 'int', '여객처리' : 'int', '복합원인' : 'int', '기타' : 'int', '계' : 'int'})

# 지연율 계산 : 백분위
airports['지연율'] = (airports.계 / airports.운항) * 100
airports.head()

,공항명,운항,기상,A/C접속,A/C정비,여객처리,복합원인,기타,계,지연율
0,인천,1432905,1834,34174,3719,1130,619,23367,64843,4.525283
1,김포,679444,2110,42701,1392,998,1003,2945,51149,7.528067
2,김해,440407,1501,16764,791,606,217,1867,21746,4.937705
3,제주,809463,3221,86033,1244,598,334,1739,93169,11.509976
4,대구,107511,303,5578,89,44,7,307,6328,5.885909


In [17]:
airports.to_csv("C:/Data/airport/지연통계.csv")